In [119]:
# additional need:  
# - sort the death rate by country 
# - format result of death rate by country
# - want to pull out vaccination rate by country
# - calculate death rate pre and post vaccination 

In [146]:
import pandas as pd
import numpy as np
import csv

In [147]:
def typ_grp(Vax_Type_Cnt):
    if (Vax_Type_Cnt > 3):
        print('4+')
    else:
        print('Under 4')

In [160]:
covid_df = pd.read_csv('WHO_COVID_19_global_data.csv')
covi_df = pd.DataFrame(covid_df)
#print(covi_df)
covi_df['Vax_Typ_Grp'] = np.where(covi_df['Vax_Type_Cnt'] > 3, '4+', 'Under 4') #create new column grouping vax varieties
covi_df[['Mo', 'Day', 'Yr']] = covi_df['Date_reported'].str.split('/', expand=True) #split date field
covi_df['YrMo'] = covi_df['Yr'].map(str) + '/' + covi_df['Mo'].map(str) #concat just year and month fields
covi_df2 = covi_df.drop(['Country_code', 'WHO_region', 'Vax_Type_Cnt', 'Day', 'Mo'], axis=1) #drop uneeded columns
covi_df2.rename(columns = {'Date_reported':'Rep_Dt', 'Cumulative_deaths':'Tot_Dths', 'Cumulative_cases':'Tot_Cases', 'First_Vax_Date':'1stVaxDt'}, inplace = True) #rename columns
covi_df2 =covi_df2[["Country", "YrMo", "Rep_Dt", "New_cases", "New_deaths", "Tot_Cases", "Tot_Dths", "Vax_Rate", "1stVaxDt", "Vax_Typ_Grp"]] #reorder columns
covi_df2['Vax_PrePost'] = np.where(covi_df2['Rep_Dt'] > covi_df2['1stVaxDt'], 'Post_Vax', 'Pre_Vax') #create new column to sort data by pre/post first vax release
covi_df3 = covi_df2.loc[covi_df2['Country'] == 'Italy']
print(covi_df3)


#covi_df2.filter(covi_df2['Country']=='Italy', axis=0)
#covi_df2['New_Death_Rate'] = covi_df2['New_deaths']/covi_df2['New_cases']
#covi_df3 = covi_df2.loc[covi_df2['Day'] == '15'] #make dataset monthly
#covi_df3

      Country    YrMo     Rep_Dt  New_cases  New_deaths  Tot_Cases  Tot_Dths  \
93112   Italy  2020/1   1/3/2020          0           0          0         0   
93113   Italy  2020/1   1/4/2020          0           0          0         0   
93114   Italy  2020/1   1/5/2020          0           0          0         0   
93115   Italy  2020/1   1/6/2020          0           0          0         0   
93116   Italy  2020/1   1/7/2020          0           0          0         0   
...       ...     ...        ...        ...         ...        ...       ...   
94011   Italy  2022/6  6/20/2022      34255          18   17879160    167721   
94012   Italy  2022/6  6/21/2022      16905          59   17896065    167780   
94013   Italy  2022/6  6/22/2022      63264          62   17959329    167842   
94014   Italy  2022/6  6/23/2022      54873          50   18014202    167892   
94015   Italy  2022/6  6/24/2022      57432          75   18071634    167967   

       Vax_Rate    1stVaxDt Vax_Typ_Grp

In [174]:
cases_deaths_by_country = covi_df2.groupby(['Country'])[['New_cases', 'New_deaths']].apply(sum) # added up new cases and new deaths and grouped by country
print(cases_deaths_by_country) #results - used to find death rate as well

                                                    New_cases  New_deaths
Country                                                                  
Afghanistan                                            181934        7715
Albania                                                278211        3497
Algeria                                                266015        6875
American Samoa                                           6414          31
Andorra                                                 43774         153
...                                                       ...         ...
Wallis and Futuna                                         454           7
Yemen                                                   11824        2149
Zambia                                                 324334        3997
Zimbabwe                                               254919        5536
occupied Palestinian territory, including east ...     658503        5660

[237 rows x 2 columns]


In [175]:
death_rate_by_country = cases_deaths_by_country['New_deaths'] / cases_deaths_by_country['New_cases'] #calculated death rate based on deaths/total cases
print(death_rate_by_country)
# need to sort to show highest and lowest

Country
Afghanistan                                                 0.042405
Albania                                                     0.012570
Algeria                                                     0.025844
American Samoa                                              0.004833
Andorra                                                     0.003495
                                                              ...   
Wallis and Futuna                                           0.015419
Yemen                                                       0.181749
Zambia                                                      0.012324
Zimbabwe                                                    0.021717
occupied Palestinian territory, including east Jerusalem    0.008595
Length: 237, dtype: float64


In [103]:
Covid_Tbl = pd.pivot_table(covi_df3, values=['New_cases','New_deaths'], index=['YrMo'],
                     aggfunc=np.sum, fill_value=0)
Covid_Tbl

,New_cases,New_deaths
YrMo,,
2020/1,3,0
2020/10,334663,2446
2020/11,937504,16583
2020/12,498511,18700
2020/2,885,21
2020/3,100851,11570
2020/4,101852,16091
2020/5,29073,5658
2020/6,7772,1404


In [104]:
Covid_Tbl2 = pd.pivot_table(covi_df2, values=['New_cases','New_deaths'], index=['YrMo'], columns=['Vax_Typ_Grp'],
                     aggfunc=np.sum, fill_value=0)
Covid_Tbl2

New_cases           New_deaths        
Vax_Typ_Grp        4+   Under 4         4+ Under 4
YrMo                                              
2020/1           9850         5        213       0
2020/10       8882198   3520164     139107   42905
2020/11      10553542   6507692     208255   75785
2020/12       9515341   9554072     231767  131281
2020/2          74667       859       2715       7
2020/3         500758    246724      33247    7485
2020/4        1055322   1257710     106229   88051
2020/5        1621041   1276004      97866   62332
2020/6        2923848   1360881     116414   36863
2020/7        4319206   2727663     142056   43042
2020/8        5981983   2330067     148200   45676
2020/9        6609875   2109821     135635   33364
2021/1       10041292   9724827     249072  180611
2021/10       7153534   6210841     119554   97241
2021/11       9516790   5892431     134052   81707
2021/12      13421568  11297984     126842   89819
2021/2        7073510   3975418     190940  110865
2021/3       11180856   3262917     222067   59324
2021/4       17894340   4126746     324818   47509
2021/5       17993573   2208351     339390   43522
2021/6        9441020   1992409     235002   34474
2021/7       11112653   4698702     209857   59373
2021/8       12012855   7795451     217470   90349
2021/9        8560861   7579159     151151  106255
2022/1       57581056  32079568     135241  109414
2022/2       43381893  15325814     174688  105384
2022/3       43677938   6841784     127777   56436
2022/4       22061516   3199695      57353   29725
2022/5       12306367   3797208      31145   17848
2022/6        9214692   3318946      18378   10964

In [105]:
#Question 1: Which country or countries has the highest number of vaccine types?
#Question 2: Which country or countries has the lowest number of vaccine types?
Cov_Qs = covi_df.groupby('Country')['Vax_Type_Cnt'].mean()
Cov_Q1_Q2 = pd.DataFrame(Cov_Qs)
Cov_Q1 = Cov_Q1_Q2.sort_values(by='Vax_Type_Cnt', ascending=False)
Cov_Q2 = Cov_Q1_Q2.sort_values(by=['Vax_Type_Cnt'])

In [106]:
#Question 1 Answer
print('Country with the highest number of vaccine types:')
Cov_Q1.head(1)

Country with the highest number of vaccine types:


,Vax_Type_Cnt
Country,
Iran (Islamic Republic of),12.0


In [107]:
#Question 2 Answer
print('Countries with the fewest number of vaccine types:')
Cov_Q2.head()

Countries with the fewest number of vaccine types:


,Vax_Type_Cnt
Country,
Saba,0.0
Sint Eustatius,0.0
Bonaire,0.0
Montserrat,1.0
Chad,1.0
